In [7]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate


In [46]:
# Load embeddings (assuming they are already loaded)
df = pd.read_csv('./combined_codes_updated.csv')

embeddings = torch.load('old_embeddings.pt')
embeddings2 = torch.load('fixdd_embeddings.pt')
labels = df['SmellKey'].values
print(labels)
print(embeddings.shape, embeddings2.shape, labels.shape)
# Combine embeddings into pairs
X = torch.stack((embeddings, embeddings2), dim=1)  # Shape: (N, 2, 768)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [73]:
# Define the model, addressing the dropout warning
class TwoTimestepLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)  # Use hidden state from the last timestep
        output = self.fc(hidden[-1])
        return output

# Create model instance
model = TwoTimestepLSTM(768, 256, 1 )  # Adjust hyperparameters len(labels.unique())

# Loss and optimizer

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

# Training loop with early stopping and validation set
num_epochs = 50  # Increase epochs if needed
batch_size = 32
best_val_acc = 0
patience = 5  # Number of epochs to wait for improvement before stopping



In [98]:
from torch.utils.data import DataLoader, TensorDataset
# Ensure y_train is converted to a tensor if it's a NumPy array
y_train = torch.tensor(y_train, dtype=torch.float32)  # Adjust dtype as needed
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Adjust dtype as needed
# Create the TensorDataset
train_data = TensorDataset(X_train, y_train)
# Define empty lists to store evaluation metrics
train_accuracy_list = []
val_accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

print(X_train.shape, y_train.dtype)
for epoch in range(num_epochs):
    train_data = TensorDataset(X_train, y_train)
    #train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    train_loader = DataLoader(train_data, batch_size=5, shuffle=True)

    val_loss = 0
    val_acc = 0

    # Training within the loops
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        loss_function = nn.BCELoss()
        labels = labels.unsqueeze(1)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
            # Evaluate on validation set after each epoch
        val_accuracy, val_precision, val_recall, val_f1 = evaluate(model,train_loader )
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy}, '
            f'Validation Precision: {val_precision}, Validation Recall: {val_recall}, '
            f'Validation F1 Score: {val_f1}')
        



C:\Users\huday\AppData\Local\Temp\ipykernel_21024\737952401.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)  # Adjust dtype as needed
C:\Users\huday\AppData\Local\Temp\ipykernel_21024\737952401.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Adjust dtype as needed


torch.Size([480, 2, 768]) torch.float32
Epoch 1/50, Validation Accuracy: 3.05, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 3.0, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 2.9583333333333335, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 3.066666666666667, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 2.9, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 3.05, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 2.9833333333333334, Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 1/50, Validation Accuracy: 2.8916666666666666, Validation Precision: 1.0, Validation Recall: 1.0, Validation F

KeyboardInterrupt: 